# <3주차 MLstudy 강의정리 및 코드실습>

## Lab6-1: Softmax Classifier

### sample dataset을 사용하여 여러개의 class를 예측할 때 사용하는 Softmax function 코드 작성

지금까지는 이진 분류(O,X)를 다뤘지만 softmax는 세개 이상의 class로 분류한다.

결과값을 확률로 나눠준다. 각 확률들의 합은 항상 1이다. 결국 가장 높은 확률을 가진 범주로 데이터가 분류된다.

H(X)=WX 가 100,200.-10 의 값이 있으면 0과 1로 고르기에 적합하지 않다. 따라서 softmax를 사용해서 class를 나눠준다.

XW=Y logistic classifier

2.0

1.0

0.1 

hypothesis = tf.nn.softmax(tf.matmul(X,W)+b)

다음 3개의 값들이 S(yi) softmax function을 거쳐 확률로 나타난다.

0.7

0.2

0.1

ex) 동물구별, 인물구별 등등

In [35]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

tf.random.set_seed(777)

2.0.0


In [36]:
# sample dataset
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

#convert into numpy and float format
x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)

In [37]:
#dataset을 선언합니다.
# dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data))
# dataset = dataset.repeat().batch(2)

nb_classes = 3 #class의 개수입니다. 3개의 클래스로 만듬

print(x_data.shape)
print(y_data.shape)

(8, 4)
(8, 3)


In [38]:
#Weight and bias setting
W = tf.Variable(tf.random.normal((4, nb_classes)), name='weight')
b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
variables = [W, b]

print(W,b)

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[ 0.7706481 ,  0.37335402, -0.05576323],
       [ 0.00358377, -0.5898363 ,  1.5702795 ],
       [ 0.2460895 , -0.09918973,  1.4418385 ],
       [ 0.3200988 ,  0.526784  , -0.7703731 ]], dtype=float32)> <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([-1.3080608 , -0.13253094,  0.5513761 ], dtype=float32)>


In [39]:
# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
def hypothesis(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)

print(hypothesis(x_data))

tf.Tensor(
[[1.36571955e-02 7.90162385e-03 9.78441238e-01]
 [3.92598137e-02 1.70347560e-02 9.43705440e-01]
 [3.80385250e-01 1.67723224e-01 4.51891541e-01]
 [3.23390484e-01 5.90759404e-02 6.17533624e-01]
 [3.62997412e-06 6.20727292e-08 9.99996305e-01]
 [2.62520462e-02 1.07279727e-02 9.63019967e-01]
 [1.56525111e-05 4.21802781e-07 9.99983907e-01]
 [2.94077745e-06 3.81134022e-08 9.99997020e-01]], shape=(8, 3), dtype=float32)


In [40]:
# Softmax onehot test
sample_db = [[8,2,1,4]]
sample_db = np.asarray(sample_db, dtype=np.float32)

# 출력 첫번째 값이 평균, 합은 1
print(hypothesis(sample_db))

tf.Tensor([[0.9302204  0.06200533 0.00777428]], shape=(1, 3), dtype=float32)


In [41]:
def cost_fn(X, Y):
    logits = hypothesis(X)
    cost = -tf.reduce_sum(Y * tf.math.log(logits), axis=1)
    cost_mean = tf.reduce_mean(cost)
    
    return cost_mean # cost 평균

print(cost_fn(x_data, y_data))

tf.Tensor(6.07932, shape=(), dtype=float32)


In [42]:
x = tf.constant(3.0)
with tf.GradientTape() as g:
    g.watch(x)
    y = x * x # x^2
dy_dx = g.gradient(y, x) # Will compute to 6.0
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


In [43]:
def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables) # W,b 를 의미

        return grads

print(grad_fn(x_data, y_data))

[<tf.Tensor: id=371783, shape=(4, 3), dtype=float32, numpy=
array([[ 0.06914611, -0.6509784 ,  0.5818323 ],
       [-1.5221257 , -1.214863  ,  2.7369888 ],
       [-1.2473828 , -1.7611002 ,  3.008483  ],
       [-1.2014606 , -1.865923  ,  3.0673838 ]], dtype=float32)>, <tf.Tensor: id=371782, shape=(3,), dtype=float32, numpy=array([-0.15212911, -0.342192  ,  0.4943211 ], dtype=float32)>]


In [44]:
def fit(X, Y, epochs=2000, verbose=100): #2000번 동안
    optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1)

    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
            print('Loss at epoch %d: %f' %(i+1, cost_fn(X, Y).numpy()))
            
fit(x_data, y_data)  # 결과는 loss가 줄어드는 방향으로 나온다.

Loss at epoch 1: 2.849417
Loss at epoch 100: 0.684151
Loss at epoch 200: 0.613813
Loss at epoch 300: 0.558204
Loss at epoch 400: 0.508305
Loss at epoch 500: 0.461059
Loss at epoch 600: 0.415072
Loss at epoch 700: 0.369636
Loss at epoch 800: 0.324533
Loss at epoch 900: 0.280721
Loss at epoch 1000: 0.246752
Loss at epoch 1100: 0.232798
Loss at epoch 1200: 0.221645
Loss at epoch 1300: 0.211476
Loss at epoch 1400: 0.202164
Loss at epoch 1500: 0.193606
Loss at epoch 1600: 0.185714
Loss at epoch 1700: 0.178415
Loss at epoch 1800: 0.171645
Loss at epoch 1900: 0.165350
Loss at epoch 2000: 0.159483


In [45]:
sample_data = [[2,1,3,2]] # answer_label [[0,0,1]]
sample_data = np.asarray(sample_data, dtype=np.float32)

a = hypothesis(sample_data) # a가 가장 높은 값으로 지정

print(a)
print(tf.argmax(a, 1)) #index: 2

tf.Tensor([[0.00112886 0.08154671 0.9173244 ]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)


In [46]:
b = hypothesis(x_data)
print(b)
print(tf.argmax(b, 1))
print(tf.argmax(y_data, 1)) # matches with y_data

tf.Tensor(
[[2.1975952e-06 1.2331187e-03 9.9876475e-01]
 [1.1288583e-03 8.1546709e-02 9.1732442e-01]
 [2.2205577e-07 1.6418636e-01 8.3581340e-01]
 [6.3921816e-06 8.5045439e-01 1.4953929e-01]
 [2.6150790e-01 7.2644752e-01 1.2044572e-02]
 [1.3783246e-01 8.6214006e-01 2.7417687e-05]
 [7.4242145e-01 2.5754160e-01 3.6978621e-05]
 [9.2197549e-01 7.8023978e-02 6.0006039e-07]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)


In [47]:
class softmax_classifer(tf.keras.Model):
    def __init__(self, nb_classes):
        super(softmax_classifer, self).__init__()
        self.W = tf.Variable(tf.random.normal((4, nb_classes)), name='weight')
        self.b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
        
    def softmax_regression(self, X):
        return tf.nn.softmax(tf.matmul(X, self.W) + self.b)
    
    def cost_fn(self, X, Y):
        logits = self.softmax_regression(X)
        cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.math.log(logits), axis=1))        
        return cost
    
    def grad_fn(self, X, Y):
        with tf.GradientTape() as tape:
            cost = self.cost_fn(x_data, y_data)
            grads = tape.gradient(cost, self.variables)            
            return grads
    
    def fit(self, X, Y, epochs=2000, verbose=500):
        optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1)

        for i in range(epochs):
            grads = self.grad_fn(X, Y)
            optimizer.apply_gradients(zip(grads, self.variables))
            if (i==0) | ((i+1)%verbose==0):
                print('Loss at epoch %d: %f' %(i+1, self.cost_fn(X, Y).numpy()))
            
model = softmax_classifer(nb_classes)
model.fit(x_data, y_data)

Loss at epoch 1: 2.472669
Loss at epoch 500: 0.375229
Loss at epoch 1000: 0.229923
Loss at epoch 1500: 0.182147
Loss at epoch 2000: 0.150633


## Lab6-2: Softmax Classifier (Animal classification)

Softmax를 사용하여 동물 데이터(zoo data.csv) 분류

In [48]:
tf.random.set_seed(777)  # for reproducibility

In [49]:
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1] #마지막 열을 제외한 모든 열 포함
y_data = xy[:, -1] #마지막 열을 선택

nb_classes = 7  # 0 ~ 6

# Make Y data as onehot shape
Y_one_hot = tf.one_hot(y_data.astype(np.int32), nb_classes) #7 one hot shape (?,1,7)

print(x_data.shape, Y_one_hot.shape)

(101, 16) (101, 7)


In [50]:
#Weight and bias setting
W = tf.Variable(tf.random.normal((16, nb_classes)), name='weight')
b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
variables = [W, b]

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
def logit_fn(X):
    return tf.matmul(X, W) + b

def hypothesis(X):
    return tf.nn.softmax(logit_fn(X))

def cost_fn(X, Y):
    logits = logit_fn(X)
    cost_i = tf.keras.losses.categorical_crossentropy(y_true=Y, y_pred=logits, 
                                                      from_logits=True)    
    cost = tf.reduce_mean(cost_i)    
    return cost

def grad_fn(X, Y): #gradient descent
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)
        return grads
    
def prediction(X, Y): #우리가 얼마나 정답을 잘 맞추고 있는지 확인하는 함수
    pred = tf.argmax(hypothesis(X), 1)
    correct_prediction = tf.equal(pred, tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    return accuracy

In [51]:
def fit(X, Y, epochs=1000, verbose=100):
    optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1)

    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
#print('Loss at epoch %d: %f' %(i+1, cost_fn(X, Y).numpy()))
            acc = prediction(X, Y).numpy()
            loss = cost_fn(X, Y).numpy() 
            print('Steps: {} Loss: {}, Acc: {}'.format(i+1, loss, acc))

fit(x_data, Y_one_hot) #결국 마지막 결과는 100% 정확도를 보여줌

Steps: 1 Loss: 3.6350293159484863, Acc: 0.1683168262243271
Steps: 100 Loss: 0.5194158554077148, Acc: 0.7920792102813721
Steps: 200 Loss: 0.31850093603134155, Acc: 0.9108911156654358
Steps: 300 Loss: 0.23534879088401794, Acc: 0.9405940771102905
Steps: 400 Loss: 0.18872138857841492, Acc: 0.9504950642585754
Steps: 500 Loss: 0.1584603488445282, Acc: 0.9504950642585754
Steps: 600 Loss: 0.13703754544258118, Acc: 0.9900990128517151
Steps: 700 Loss: 0.12098982185125351, Acc: 0.9900990128517151
Steps: 800 Loss: 0.10847964882850647, Acc: 1.0
Steps: 900 Loss: 0.09843041002750397, Acc: 1.0
Steps: 1000 Loss: 0.09016558527946472, Acc: 1.0
